In [1]:
import utils
from optparse import OptionParser
import os
import compress_pickle as pickle
from tqdm import tqdm
import multiprocessing as mp
import numpy as np
import pandas as pd
import copy

save_loc="/gscratch/prl/wagnew3/microsoft_academic_graph/"
patents_save_loc="/gscratch/prl/wagnew3/microsoft_academic_graph/patents/"
data_dir="/gscratch/prl/wagnew3/microsoft_academic_graph/"
num_papers=-1

In [2]:
if True:#not os.path.exists(os.path.join(save_loc, 'cvpr_patents_graph_crawl_list.lz4')):
    loaded_dict=utils.load_mag(data_dir, n_papers=num_papers)
    cvpr_graph=pickle.load(open(os.path.join(data_dir, f'cvpr_graph_-1.lz4'), 'rb'))

    ''' Save list of patents directly citing papers from conferences.

    args:
        loaded_dict: dict of MAG dataframes from load_mag
        subgraph: subgraph with MAGS to get patents for in keys
    returns:
        patents, dict MAG : citing patents and metadata
    '''



    subgraph=cvpr_graph
    all_mags=list(subgraph.keys())
    for patents in list(subgraph.values()):
        all_mags+=patents
    all_mags=set(all_mags)
    print(f'len(all_mags) {len(all_mags)}')
    papers_patents=loaded_dict['_pcs_mag_doi_pmid.tsv']
    print(f'{len(papers_patents)} paper patent linkages loaded')
    papers_patents_old=loaded_dict['_pcs.tsv']
    paper_to_patent={}
    patents=[]

    for row_ind in tqdm(range(len(papers_patents))):
        # https://onlinelibrary.wiley.com/doi/10.1111/jems.12455 Table 1, 0.63% error
        paper_mag=str(papers_patents['magid'][row_ind])
        if paper_mag in all_mags:
            patent_id=str(papers_patents['patent'][row_ind])
            if paper_mag not in paper_to_patent:
                paper_to_patent[paper_mag]=[patent_id]
            else:
                paper_to_patent[paper_mag].append(patent_id)
            patents.append(patent_id)

    # for row_ind in tqdm(range(len(papers_patents_old))):
    #     paper_mag=str(papers_patents_old['magid'][row_ind])
    #     if paper_mag in all_mags:
    #         patent_id=str(papers_patents_old['patent'][row_ind])
    #         if paper_mag not in paper_to_patent:
    #             paper_to_patent[paper_mag]=[patent_id]
    #         else:
    #             paper_to_patent[paper_mag].append(patent_id)
    #         patents.append(patent_id)

    # Get list of unique patents
    patents_list=list(set(patents))

    pickle.dump(patents_list, open(os.path.join(save_loc, 'cvpr_patents_graph_crawl_list.lz4'), 'wb'))
else:
    patents_list=pickle.load(open(os.path.join(save_loc, 'cvpr_patents_graph_crawl_list.lz4'), 'rb'))
    
print(f'downloading {len(patents_list)} patents')

  0%|                                                                                | 0/42 [00:00<?, ?it/s]

loading Authors.txt
loading from cached file /gscratch/prl/wagnew3/microsoft_academic_graph/cached_Authors_-1.lz4


  2%|█▋                                                                   | 1/42 [02:59<2:02:31, 179.31s/it]

loading PaperFieldsOfStudy.txt
loading from cached file /gscratch/prl/wagnew3/microsoft_academic_graph/cached_PaperFieldsOfStudy_-1.lz4


  7%|████▉                                                                | 3/42 [12:56<2:54:01, 267.74s/it]

...

loading _pcs_mag_doi_pmid.tsv
loading from cached file /gscratch/prl/wagnew3/microsoft_academic_graph/cached__pcs_mag_doi_pmid_-1.lz4


100%|███████████████████████████████████████████████████████████████████████| 42/42 [17:42<00:00, 25.29s/it]


loaded mag
len(all_mags) 39942894
40393300 paper patent linkages loaded


100%|███████████████████████████████████████████████████████| 40393300/40393300 [04:23<00:00, 153515.72it/s]


downloading 2632492 patents


In [ ]:
# Get metadata for all patents
from concurrent.futures import ThreadPoolExecutor
from multiprocessing import Pool

patents_info_save_loc=os.path.join(save_loc, 'patents')
all_parallel_runs=[]
patents_scraped_info={}
patents_list=patents_list[:10000]

scrape_args=[]
for patent in tqdm(patents_list):
    #patents_scraped_info.update(utils.scrape_patents([patent], patents_save_loc))
    scrape_args.append(([patent], patents_save_loc, False, True))

# # for i in range(10):
# #     utils.scrape_patents(scrape_args[i])
    
# with ThreadPoolExecutor(max_workers=80) as p:
#     results=p.map(utils.scrape_patents, scrape_args)
#     print(f'len(results {len(results)}')
#     for result in tqdm(results):
#         u=0
        
with Pool(processes=64) as pool, tqdm(total=len(scrape_args)) as pbar: # create Pool of processes (only 2 in this example) and tqdm Progress bar
    for data in pool.imap_unordered(utils.scrape_patents, scrape_args):                   # send urls from all_urls list to parse() function (it will be done concurently in process pool). The results returned will be unordered (returned when they are available, without waiting for other processes)
        pbar.update()                           

# pool = mp.Pool(processes=80, maxtasksperchild=1)
# for patent in tqdm(patents_list):
#     #patents_scraped_info.update(utils.scrape_patents([patent], patents_save_loc))
#     run=pool.apply_async(utils.scrape_patents, args=([patent], patents_save_loc, False, True))
#     all_parallel_runs.append(run)

# scrapped_succesfully=0
# num_runs=len(all_parallel_runs)
# for run_num in tqdm(range(num_runs)):
#     run=all_parallel_runs[run_num]
#     all_parallel_runs[run_num]=None
#     run.get()
    
    
#     patent_id=list(ret_dict.keys())[0]
#     patent_info=ret_dict[patent_id]
#     if 'title' in patent_info:
#         scrapped_succesfully+=1
#         patent_info_row=[patent_id, patent_info['title'], patent_info['inventor_name'], patent_info['assignee_name_orig'], patent_info['assignee_name_current'],
#                          patent_info['pub_date'], patent_info['priority_date'], patent_info['grant_date'], patent_info['filing_date'],
#                          patent_info['abstract_text'], patent_info['patent_full_url'], patent_info['classification_code_descss'], patent_info['patent_text']] 

#         pickle.dump(patents_scraped_info, open(os.path.join(patents_info_save_loc, f'{patent_id}.lz4'), 'wb'))

        
#patents_scraped_info=utils.scrape_patents(patents_list)


# patents_info_csv=[['patent_id',
#                    'title',
#                    'inventor_name',
#             'assignee_name_orig',
#             'assignee_name_current',
#             'pub_date',
#             'priority_date',
#             'grant_date',
#             'filing_date',
#             'abstract_text',
#             'patent_full_url']]


# scrapped_succesfully=0
# for patent_id in patents_scraped_info:
#     patent_info=patents_scraped_info[patent_id]
#     # patent_pdf=requests.get()



#     # with open(, 'w') as f:
#     #     f.write(patent_pdf)
#     if 'title' in patent_info:
#         scrapped_succesfully+=1
#         patent_info_row=[patent_id, patent_info['title'], patent_info['inventor_name'], patent_info['assignee_name_orig'], patent_info['assignee_name_current'],
#                          patent_info['pub_date'], patent_info['priority_date'], patent_info['grant_date'], patent_info['filing_date'],
#                          patent_info['abstract_text'], patent_info['patent_full_url'], patent_info['classification_code_descss'], patent_info['patent_text']] 
#     patents_info_csv.append(patent_info_row )

print(f'saved {len(patents_info_csv)} patents')

# with open(os.path.join(save_loc, patents_file_name+'_patents_metadata.tsv'), 'w', newline='') as csvfile:
#     csvwriter = csv.writer(csvfile, delimiter='\t', quotechar='|', quoting=csv.QUOTE_ALL)
#     csvwriter.writerows(patents_info_csv)

In [6]:
patents_info_dict={}
agg_data_dir="/gscratch/scrubbed/wagnew3/data/"
for agg_file in tqdm(os.listdir(agg_data_dir)):
    if "patent_agg" in agg_file:
        patent_info_sub=pickle.load(os.path.join(agg_data_dir, agg_file))
        patents_info_dict.update(patent_info_sub)
print(f"{len(patents_info_dict)} patents")

#make MAG ID to conference and journal id dicts
subgraph=pickle.load(open(os.path.join(data_dir, f'cvpr_graph_-1.lz4'), 'rb'))
mag_to_conf_id={}
mag_to_journal_id={}
mag_to_title={}
conf_save_path=os.path.join(save_loc, 'mag_conf_journ_title.lz4')
if not os.path.exists(conf_save_path):
    papers=loaded_dict["Papers.txt"]
    for paper_ind in tqdm(range(0, len(papers))):
        mag_id=str(int(papers['PaperId'][paper_ind]))
        mag_to_conf_id[mag_id]=str(papers['ConferenceSeriesId'][paper_ind])
        mag_to_journal_id[mag_id]=str(papers['JournalId'][paper_ind])
        title=str(papers['PaperTitle'][paper_ind])
        mag_to_title[mag_id]=title
    pickle.dump((mag_to_conf_id, mag_to_journal_id, mag_to_title), open(conf_save_path, 'wb'))
else:
    mag_to_conf_id, mag_to_journal_id, mag_to_title=pickle.load(conf_save_path)

100%|███████████████████████████████████████████████████████████████████████| 26/26 [07:23<00:00, 17.04s/it]


2549890 patents


In [8]:
def get_conference_keywords(patents_info_dict, stored_patents_keywords_dict, patent_id, keywords, 
                            depth_vec, depth, citing_patents_depth_zero, paper_mag):
    patent_id=patent_id.replace("-", "")
    if not patent_id in stored_patents_keywords_dict:
        stored_patents_keywords_dict[patent_id]={}
        for keyword_ind in range(len(keywords)):
            keyword=keywords[keyword_ind]
            stored_patents_keywords_dict[patent_id][keyword]=0 
            #if patent_id not in patent_keywords_dict:
                # Search patent for keywords
            keyword_in_patent=0
            if patent_id in patents_info_dict:
                patent_info=patents_info_dict[patent_id]
                if isinstance(patent_info, list):
                    if keyword in patent_info[1] or keyword in patent_info[12]:
                        stored_patents_keywords_dict[patent_id][keyword]=1
                    else:
                        stored_patents_keywords_dict[patent_id][keyword]=0
    
    for keyword_ind in range(len(keywords)):
        keyword=keywords[keyword_ind]
        if stored_patents_keywords_dict[patent_id][keyword]==1:
            depth_vec[keyword_ind,0,depth]+=1
            if depth==0:
                citing_patents_depth_zero[paper_mag][keyword].append(patent_id)
        else:
            depth_vec[keyword_ind,1,depth]+=1


#compute distribution of citation distance to keyword
dist_to_keyword=np.zeros(1000000+1)
#CVPR
conference_series_ids=['1158167855']


def update_from_stored(mag_id, patent_keywords_dict, depth_vec, 
                       patent_in_linkage_count, paper_in_linkage_count,
                      store_depth, depth):
    '''
    Update keyword search with stored search info from mag_id.
    '''
    store_depth_vec, store_papers, store_patents=patent_keywords_dict[mag_id]
    # if paper_in_linkage_count is not None:
    #     # Update patent counts
    #     for store_patent_id in store_patents:
    #         if store_patent_id not in patent_in_linkage_count:
    #             patent_in_linkage_count[store_patent_id]=0
    #         patent_in_linkage_count[store_patent_id]+=store_patents[store_patent_id]
    #     # Update paper counts
    #     for store_mag_id in store_papers:
    #         if store_mag_id not in paper_in_linkage_count:
    #             paper_in_linkage_count[store_mag_id]=0
    #         paper_in_linkage_count[store_mag_id]+=store_papers[store_mag_id]
    # Update depth vec with stored keyword counts
    depth_vec[:,:,depth:]+=store_depth_vec
    return depth_vec

def find_keyword(subgraph, mag_id, patent_keywords_dict, patents_info_dict, 
                 depth, keywords, paper_in_linkage_count, patent_in_linkage_count,
                 stored_patents_keywords_dict, citing_patents_depth_zero,
                 max_depth, store_depth=2, visted_patents=None, depth_vec=None,
                 mag_id_keyword_depth_vecs=None):
    '''
    paper_in_linkage_count: number of times a paper is in a citation chain
    patent_in_linkage_count: number of times a patent is in citation chain
    '''
    depth_vec=np.zeros((len(keywords),2,max_depth))#{keyword:np.zeros((2,max_depth)) for keyword in keywords}#np.zeros((2,max_depth))
    if depth==0:
        citing_patents_depth_zero[mag_id]={keyword:[] for keyword in keywords}
    if depth>=max_depth:
        return depth_vec
    # if depth_vec is None:
    #     # vector of citation depth at which patent with keyword is found
    #     depth_vec=np.zeros(max_depth)
    
    #Check if results are stored
    if max_depth-depth==store_depth and mag_id in patent_keywords_dict:
        depth_vec=update_from_stored(mag_id, patent_keywords_dict, depth_vec, 
                                     patent_in_linkage_count, paper_in_linkage_count,
                                    store_depth, depth)
        #print("cached")
        return depth_vec
    elif max_depth-depth==store_depth:
        # If search is deep enough to cache results, create cache data structures
        copy_depth_vec=np.copy(depth_vec)
        depth_vec=np.zeros((len(keywords),2,max_depth))#{keyword:np.zeros((2,max_depth)) for keyword in keywords}
        copy_paper_in_linkage_count=paper_in_linkage_count
        copy_patent_in_linkage_count=patent_in_linkage_count
        paper_in_linkage_count={}
        patent_in_linkage_count={}
        
    # Check if any citing patents contain keyword
    if mag_id in paper_to_patent:
        prev_sub_keyword_depth=np.copy(depth_vec)
        citing_patents=paper_to_patent[mag_id]
        # print('citing_patents', citing_patents)
        # if visted_patents is not None:
        #     visted_patents+=citing_patents
        for citing_patent in citing_patents:
            get_conference_keywords(patents_info_dict, stored_patents_keywords_dict, citing_patent, keywords, 
                                        depth_vec, depth, citing_patents_depth_zero, mag_id)
            # if res==1:
            #     depth_vec[0,depth]+=1
            #     if patent_in_linkage_count is not None:
            #         if citing_patent not in patent_in_linkage_count:
            #             patent_in_linkage_count[citing_patent]=0
            #         patent_in_linkage_count[citing_patent]+=1
            # else:
            #     depth_vec[1,depth]+=1
        # if paper_in_linkage_count is not None:
        #     if mag_id not in paper_in_linkage_count:
        #         paper_in_linkage_count[mag_id]=0
        #     paper_in_linkage_count[mag_id]+=np.sum(depth_vec[0])-np.sum(prev_sub_keyword_depth)

    min_depth=1000000
    if depth+1<max_depth and mag_id in subgraph:
        for sub_mag_id in subgraph[mag_id]:
            # print(f"sub {sub_mag_id}")
            sub_depth_vec=find_keyword(subgraph, sub_mag_id, patent_keywords_dict, 
                                   patents_info_dict, depth+1, keywords,
                                   paper_in_linkage_count, patent_in_linkage_count, stored_patents_keywords_dict,
                                   citing_patents_depth_zero, max_depth=max_depth, visted_patents=visted_patents, 
                                   depth_vec=depth_vec, mag_id_keyword_depth_vecs=mag_id_keyword_depth_vecs)
            # print(depth_vec, sub_depth_vec)
            depth_vec+=sub_depth_vec
            # if paper_in_linkage_count is not None:
            #     if mag_id not in paper_in_linkage_count:
            #         paper_in_linkage_count[mag_id]=0
            #     paper_in_linkage_count[mag_id]+=np.sum(sub_depth_vec[0])
    
    if max_depth-depth==store_depth:
        # Update with cached data structures if caching
        
        store_depth_vec=np.copy(depth_vec)[:,:,depth:]
        # for keyword in depth_vec:
        #     store_depth_vec[keyword]=np.copy(depth_vec[keyword])[:,depth:]
        # print("store_depth_vec", store_depth_vec)
        patent_keywords_dict[mag_id]=(store_depth_vec, paper_in_linkage_count, patent_in_linkage_count)
        paper_in_linkage_count=copy_paper_in_linkage_count
        patent_in_linkage_count=copy_patent_in_linkage_count
        depth_vec=copy_depth_vec
        depth_vec=update_from_stored(mag_id, patent_keywords_dict, depth_vec, 
                                     patent_in_linkage_count, paper_in_linkage_count,
                                    store_depth, depth)

    if mag_id in mag_id_keyword_depth_vecs:
        if len(mag_id_keyword_depth_vecs[mag_id][0,0])<max_depth-depth:
            mag_id_keyword_depth_vecs[mag_id]=depth_vec[:,:,depth:]
    else:
        mag_id_keyword_depth_vecs[mag_id]={}
        mag_id_keyword_depth_vecs[mag_id]=depth_vec[:,:,depth:]
        
    return depth_vec

#Singlethreaded
def analyze_graph(keywords):
    keywords=keywords
    max_depth=3
    # How many patents with keyword citing paper at cite dist
    cited_count_vec=np.zeros((2,max_depth))
    # If any patents with keyword citing paper at cite dist
    cited_pres_vec=np.zeros((2,max_depth))
    paper_in_linkage_count={}
    patent_in_linkage_count={}
    patent_keywords_dict={}
    stored_patents_keywords_dict={keyword:{} for keyword in keywords}
    mag_id_keyword_depth_vecs={}
    citing_patents_depth_zero={}
    run=0
    for mag_id in tqdm(subgraph):
        conf_id=mag_to_conf_id[mag_id]
        journal_id=mag_to_journal_id[mag_id]
        # If paper is in target venues
        if conf_id in conference_series_ids or journal_id in conference_series_ids:
            keyword_depth_vec=find_keyword(subgraph, mag_id, patent_keywords_dict, 
                                           patents_info_dict , 0, keywords, 
                                           paper_in_linkage_count, patent_in_linkage_count,
                                           stored_patents_keywords_dict, citing_patents_depth_zero,
                                           max_depth=max_depth, mag_id_keyword_depth_vecs=mag_id_keyword_depth_vecs)
            #print(keyword_depth_vec)
            mag_id_keyword_depth_vecs[mag_id]=keyword_depth_vec
            # cited_count_vec+=keyword_depth_vec
            # cited_pres_vec+=1
            # #print(f"keyword_depth {keyword_depth}")
            # if np.sum(keyword_depth_vec[0])==0:
            #     dist_to_keyword[-1]+=1
            # else:
            #     dist_to_keyword[np.argmax(keyword_depth_vec[0]!=0)]+=1
            run+=1
            # break
    print('num papers searched from', run)
    # print(dist_to_keyword, np.sum(dist_to_keyword))
    # print(dist_to_keyword[:5])
    # print(cited_count_vec, cited_pres_vec, cited_count_vec/cited_pres_vec)
    return keywords, mag_id_keyword_depth_vecs, citing_patents_depth_zero

In [5]:
affiliations=loaded_dict['Affiliations.txt']
paper_authors=loaded_dict['PaperAuthorAffiliations.txt']
paper_fields=loaded_dict['PaperFieldsOfStudy.txt']
fields=loaded_dict['FieldsOfStudy.txt']
authors=loaded_dict['Authors.txt']


# conf_save_path=os.path.join(save_loc, 'cached_affil_author_fields_dicts.lz4')
# if not os.path.exists(conf_save_path):
#     # Make dict of Affiliations
#     affiliations=loaded_dict['Affiliations.txt']
#     affiliations_dict={}
#     for ind in tqdm(range(len(affiliations))):
#         affiliations_dict[affiliations['AffiliationId'][ind]]=(affiliations['NormalizedName'][ind], affiliations['Latitude'][ind], affiliations['Longitude'][ind])

#     # # Make dict of Paper Author Affiliations
#     # paper_authors=loaded_dict['PaperAuthorAffiliations.txt']
#     # paper_author_dict={}
#     # for ind in tqdm(range(len(paper_authors))):
#     #     paper_id=paper_authors['PaperId'][ind]
#     #     author_id=paper_authors['AuthorId'][ind]
#     #     affil_id=paper_authors['AffiliationId'][ind]
#     #     if paper_id not in paper_author_dict:
#     #         paper_author_dict[paper_id]=[]
#     #     paper_author_dict[paper_id].append((author_id, affil_id))

#     # Make dict of Paper Fields of Study
#     paper_fields=loaded_dict['PaperFieldsOfStudy.txt']
#     paper_feilds_dict=paper_fields.set_index('PaperId').to_dict()
#     # paper_feilds_dict={}
#     # for ind in tqdm(range(len(paper_fields))):
#     #     paper_id=paper_fields['PaperId'][ind]
#     #     field_id=paper_fields['FieldOfStudyId'][ind]
#     #     score=paper_fields['Score'][ind]
#     #     if paper_id not in paper_feilds_dict:
#     #         paper_feilds_dict[paper_id]=[]
#     #     paper_feilds_dict[paper_id].append((field_id, score))

#     # Make dict of Fields of Study
#     fields=loaded_dict['FieldsOfStudy.txt']
#     fields_dict={}
#     for ind in tqdm(range(len(fields))):
#         fields_dict[fields['FieldOfStudyId'][ind]]=(fields['NormalizedName'][ind], fields['Level'][ind])
        
#     pickle.dump((affiliations_dict, paper_author_dict, paper_feilds_dict, fields_dict), open(conf_save_path, 'wb'))
# else:
#     affiliations_dict, paper_author_dict, paper_feilds_dict, fields_dict=pickle.load(conf_save_path)

In [6]:
keywords=[["targeting", "advertisement", "ad", "purchase", "client", "consumer"],
                  ["national", "resident", "taxpayer", "citizen"],
                  ["travel", "baggage", "reidentification", "license plate", "border", "airport"],
                  ["child", "children", "kid", "minor", "youth", "age", "underage"],
                  ["criminal", "crime", "felon", "convict", "prisoner", "jail", "penitentiary", "prison", "fraud", "security", "suspect"],
                  ["army", "defense", "combat", "enemy", "security", "drone", "aircraft", "military"],
                  ["race", "ethnicity", "class", "gender", "female", "male", "woman", "man", "sexuality", "age", "sex", "caste", "disability", "nonbinary", "non-binary", "trans", "transgender"],
                  ["medical", "health", "surgery", "organ", "bone", "medicine", "doctor", "hospital", "therapy", "therapies", "drug", "treatment", "cancer", "mri", "xray", "x-ray", "tumor", "disease", "radiologist"],
                  ["iris", "irises", "face", "facial", "facial recognition", "gesture", "torso", "anatomy", "anatomies", "joint", "limb", "hand", "finger"],
                  ["person", "people", "track", "human", "pedestrian", "foot traffic"],
                  ["room", "scene", "office", "store", "street", "crowd", "home", "house", "apartment"],
                  ["location", "geolocation", "GPS", "friend", "social network", "family", "preference"],
                  ["image", "text", "object"],
                  ["local"],
                  ["wireless", "wifi", "server", "cloud", "network"],
                  ["personalize", "preference", "recommend"],
                  ["prison", "travel", "border", "criminal", "crime", "felon", "convict", "prisoner", "jail", "penitentiary", "prison", "fraud", "security"],
                  ["surveil", "surveillance"]]


flat_keywords=[]
for keyword in keywords:
    flat_keywords+=keyword

In [7]:
#!pip3 install line_profiler


    
# flat_keywords=["surveil"]
keywords, mag_id_keyword_depth_vecs, stored_patents_keywords_dict=analyze_graph(flat_keywords)

# conf_save_path=os.path.join(save_loc, 'temp_keywords_cvpr.lz4')
# pickle.dump((mag_id_keyword_depth_vecs, stored_patents_keywords_dict), open(conf_save_path, 'wb'))
# print('saved')
# %load_ext line_profiler
# %lprun -f analyze_graph -f find_keyword -f get_conference_keywords analyze_graph(flat_keywords)

# import matplotlib

# paper_chain_df = pd.DataFrame(list(paper_in_linkage_count.values()))
# paper_chain_hist = paper_chain_df.hist(bins=100, log='True')

# patent_chain_df = pd.DataFrame(list(patent_in_linkage_count.values()))
# patent_chain_df = patent_chain_df.hist(bins=100, log='True')

100%|████████████████████████████████████████████████████████| 15577957/15577957 [12:55<00:00, 20078.79it/s]


num papers searched from 19793


In [3]:
conf_save_path=os.path.join(save_loc, 'temp_keywords_cvpr.lz4')
print(conf_save_path)
mag_id_keyword_depth_vecs, stored_patents_keywords_dict=pickle.load(open(conf_save_path, 'rb'))


    

/gscratch/prl/wagnew3/microsoft_academic_graph/temp_keywords_cvpr.lz4


In [9]:
num_cvpr=0
for mag_id in mag_id_keyword_depth_vecs:
    conf_id=mag_to_conf_id[mag_id]
    if conf_id in conference_series_ids:
        num_cvpr+=1
print(num_cvpr)  

19793


In [8]:
single_keywords_dict={"paper_id":[]}
for keyword in flat_keywords:
    single_keywords_dict[keyword]=[]
    single_keywords_dict[keyword+"_0_dist_patents"]=[]

flat_keywords=list(set(flat_keywords))
for mag_id in tqdm(mag_id_keyword_depth_vecs):
    single_keywords_dict["paper_id"].append(mag_id)
    for keyword_ind in range(len(flat_keywords)):
        keyword=flat_keywords[keyword_ind]
        single_keywords_dict[keyword].append(mag_id_keyword_depth_vecs[mag_id][keyword_ind])
        if mag_id in stored_patents_keywords_dict:
            single_keywords_dict[keyword+"_0_dist_patents"].append(stored_patents_keywords_dict[mag_id][keyword])
        else:
            single_keywords_dict[keyword+"_0_dist_patents"].append([])



100%|███████████████████████████████████████████████████████████| 2070963/2070963 [18:51<00:00, 1829.56it/s]


In [12]:
conf_save_path=os.path.join(save_loc, 'temp_keywords_cvpr.lz4')
pickle.dump((mag_id_keyword_depth_vecs, stored_patents_keywords_dict), open(conf_save_path, 'wb'))

In [10]:
conf_save_path=os.path.join(save_loc, 'cvpr_df_single_keywords_dict.lz4')
pickle.dump(single_keywords_dict, open(conf_save_path, 'wb'))

In [ ]:
test_single_keywords_dict={}
test_single_keywords_dict['surveillance_0_dist_patents']=single_keywords_dict['surveillance_0_dist_patents']
df_single_keywords_dict=pd.DataFrame.from_dict(single_keywords_dict)
print('made df')
conf_save_path=os.path.join(save_loc, 'cvpr_df_single_keywords_dict.lz4')
pickle.dump(df_single_keywords_dict, open(conf_save_path, 'wb'))

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from multiprocessing import Pool

flat_keywords=[]
for keyword in keywords:
    flat_keywords+=keyword

scrape_args=[]
for keyword in tqdm(flat_keywords):
    #patents_scraped_info.update(utils.scrape_patents([patent], patents_save_loc))
    scrape_args.append(([keyword],))

res_dict={}
with Pool(processes=30) as pool, tqdm(total=len(scrape_args)) as pbar: # create Pool of processes (only 2 in this example) and tqdm Progress bar
    for data in tqdm(pool.imap_unordered(analyze_graph, scrape_args)):                   # send urls from all_urls list to parse() function (it will be done concurently in process pool). The results returned will be unordered (returned when they are available, without waiting for other processes)
        pbar.update()
        res_dict[data[0][0]]=data

conf_save_path=os.path.join(save_loc, 'temp_keywords_cvpr.lz4')
pickle.dump(res_dict, open(conf_save_path, 'wb'))

In [ ]:
# Verify analyze subgraph

for mag_id in tqdm(subgraph):
    conf_id=mag_to_conf_id[mag_id]
    journal_id=mag_to_journal_id[mag_id]
    # If paper is in target venues
    if conf_id in conference_series_ids or journal_id in conference_series_ids:
        paper_keyword_depth_vec=mag_id_keyword_depth_vecs[mag_id][0]
        sub_keyword_sum=np.zeros(4)
        for sub_mag_id in subgraph[mag_id]:
            sub_keyword_depth_vec=mag_id_keyword_depth_vecs[sub_mag_id]
            sub_keyword_sum[:len(sub_keyword_depth_vec[0])]+=sub_keyword_depth_vec[0]
        if np.sum(np.abs(paper_keyword_depth_vec[1:]-sub_keyword_sum[:-1]))!=0
            print('error, do not match!', paper_keyword_depth_vec, sub_keyword_sum)
        

In [8]:
mag_id_keyword_depth_vecs_df_dict={"paper_id":[], "keyword_depth_vec": []}
for mag_id in mag_id_keyword_depth_vecs:
    mag_id_keyword_depth_vecs_df_dict["paper_id"].append(mag_id)
    mag_id_keyword_depth_vecs_df_dict["keyword_depth_vec"].append(mag_id_keyword_depth_vecs[mag_id])
    
mag_id_keyword_depth_vecs_df=pd.DataFrame.from_dict(mag_id_keyword_depth_vecs_df_dict)
print(mag_id_keyword_depth_vecs_df.columns)
print('len(mag_id_keyword_depth_vecs_df)', len(mag_id_keyword_depth_vecs_df))

Index(['paper_id', 'keyword_depth_vec'], dtype='object')
len(mag_id_keyword_depth_vecs_df) 2070963


In [9]:
def implode(df, column):
      
    agg_keys={}
    for key in df.columns:
        if key!=column:
            agg_keys[key]=list
            
    return df.groupby(column, as_index=False).agg(agg_keys)[df.columns]

In [10]:
def validate_df(df):
    print('--------------------------------------------------------------------------------------')
    print(df.iloc[0])
    print('--------------------------------------------------------------------------------------')

In [11]:
print(len(mag_id_keyword_depth_vecs_df))

2070963


In [12]:
## Author Affils

# Merge paper author ids
paper_authors=loaded_dict['PaperAuthorAffiliations.txt']
print(mag_id_keyword_depth_vecs_df.columns)
print(paper_authors.columns)
print(len(mag_id_keyword_depth_vecs_df['paper_id']))
combined_df=mag_id_keyword_depth_vecs_df.merge(paper_authors, left_on='paper_id', right_on='PaperId', how='left', suffixes=(None, '_paper_authors'))
print("Merge paper author ids")

# Merge paper authors affils
combined_df=combined_df.explode(['AffiliationId'])
combined_df=combined_df.merge(affiliations, left_on='AffiliationId', right_on='AffiliationId', how='left', suffixes=(None, '_affiliations'))
combined_df=implode(combined_df, 'paper_id')
print("Merge paper authors affils")

full_df=mag_id_keyword_depth_vecs_df.merge(combined_df, left_on='paper_id', right_on='paper_id', how='left', suffixes=(None, '_affils'))
validate_df(full_df)
## Author Names

# Merge paper author ids
paper_authors=loaded_dict['PaperAuthorAffiliations.txt']
print(mag_id_keyword_depth_vecs_df.columns)
print(paper_authors.columns)
print(len(mag_id_keyword_depth_vecs_df['paper_id']))
combined_df=mag_id_keyword_depth_vecs_df.merge(paper_authors, left_on='paper_id', right_on='PaperId', how='left', suffixes=(None, '_paper_authors'))
combined_df=implode(combined_df, 'paper_id')
print("Merge paper author ids")

# Merge paper authors affils
combined_df=combined_df.explode(['AuthorId'])
combined_df=combined_df.merge(authors, left_on='AuthorId', right_on='AuthorId', how='left', suffixes=(None, '_author'))
combined_df=implode(combined_df, 'paper_id')
print("Merge paper authors names")

full_df=full_df.merge(combined_df, left_on='paper_id', right_on='paper_id', how='left', suffixes=(None, '_author'))
validate_df(full_df)

# Merge paper field id
combined_df=mag_id_keyword_depth_vecs_df.merge(paper_fields, left_on='paper_id', right_on='PaperId', how='left', suffixes=(None, '_paper_fields'))
print(combined_df.columns)
print(len(combined_df))
print("Merge paper field id")

# Merge paper field info
combined_df=combined_df.explode(['FieldOfStudyId'])
combined_df=combined_df.merge(fields, left_on='FieldOfStudyId', right_on='FieldOfStudyId', how='left', suffixes=(None, '_fields'))
combined_df=implode(combined_df, 'paper_id')
print("Merge paper field info")

full_df=full_df.merge(combined_df, left_on='paper_id', right_on='paper_id', how='left', suffixes=(None, '_fields'))
validate_df(full_df)

papers=loaded_dict['Papers.txt']

# Merge paper field id
combined_df=mag_id_keyword_depth_vecs_df.merge(papers, left_on='paper_id', right_on='PaperId', how='left', suffixes=(None, '_paper'))
print(combined_df.columns)
print(len(combined_df))
print("Merge paper id")

full_df=full_df.merge(combined_df, left_on='paper_id', right_on='paper_id', how='left', suffixes=(None, '_fields'))
validate_df(full_df)

# Convert and save dataset
conf_save_path=os.path.join(save_loc, 'cvpr_procssed_dataset.lz4')
pickle.dump(full_df, open(conf_save_path, 'wb'))

Index(['paper_id', 'keyword_depth_vec'], dtype='object')
Index(['PaperId', 'AuthorId', 'AffiliationId'], dtype='object')
2070963
Merge paper author ids
Merge paper authors affils
--------------------------------------------------------------------------------------
paper_id                                     193634663
keyword_depth_vec           [[[0.0, 0.0], [0.0, 0.0]]]
keyword_depth_vec_affils        [[[[0. 0.], [0. 0.]]]]
PaperId                                    [193634663]
AuthorId                                   [249105280]
AffiliationId                                    [nan]
Rank                                             [nan]
NormalizedName                                   [nan]
DisplayName                                      [nan]
GridId                                           [nan]
RorId                                            [nan]
OfficialPage                                     [nan]
WikiPage                                         [nan]
PaperCount         

In [ ]:
# Unit tests/sanity checks
import random

# How many papers in conference and dataset?
num_papers_dataset=len(loaded_dict["Papers.txt"])
print(f"There are {num_papers_dataset} papers in MAG dataset")
print(f"There are {len(all_mags)} papers in conference citation graph")

# How many patents in dataset?
num_patents_dataset=len(patents_list)
print(f"There are {num_patents_dataset} patent linkages in conference citation graph and {len(patents_info_dict)} were scrapped")

# change this number to have different things printed for manual verification
random_num=0
# Are all paper-paper linkages present?
def print_random_paper_paper_linkages(subgraph, mag_to_title, num_print=5):
    '''
    Print random paper from conference subgraph and all papers citing it to enable manual verification.
    '''
    for ind in range(num_print):
        paper_mag=random.choice(list(subgraph.keys()))
        paper_title=mag_to_title[paper_mag]
        citing_list=[]
        for citing_mag in subgraph[paper_mag]:
            citing_title=mag_to_title[citing_mag]
            citing_list.append(citing_title)
        print(f"Paper Title: {paper_title} Number Citing Papers: {len(citing_list)} Citing Papers: {citing_list}\n")
#print_random_paper_paper_linkages(subgraph, mag_to_title, num_print=5)

# Are all paper-patent linkages present?
def print_random_paper_paper_linkages(subgraph, mag_to_title, paper_to_patent, num_print=5):
    '''
    Print random paper from conference subgraph and all patents citing it to enable manual verification.
    '''
    for ind in range(num_print):
        paper_mag=random.choice(list(subgraph.keys()))
        paper_title=mag_to_title[paper_mag]
        if paper_mag in paper_to_patent:
            print(f"Paper Title: {paper_title} MAG: {paper_mag} Number Citing Patents: {len(paper_to_patent[paper_mag])} Citing Patents: {paper_to_patent[paper_mag]}\n")
#print_random_paper_paper_linkages(subgraph, mag_to_title, paper_to_patent, num_print=50)

# Is the tree search correct?
surveil_keywords=["surveil"]
max_depth=2
for mag_id in tqdm(subgraph):
    conf_id=mag_to_conf_id[mag_id]
    journal_id=mag_to_journal_id[mag_id]
    # If paper is in target venues
    patent_keywords_dict={}
    if conf_id in conference_series_ids or journal_id in conference_series_ids:
        visted_patents=[]
        keyword_vec=find_keyword(subgraph, mag_id, patent_keywords_dict, patents_info_dict, 0, surveil_keywords, visted_patents=visted_patents, max_depth=max_depth)
        print(f"Title {mag_to_title[mag_id]} keyword vec: {keyword_vec} visted_patents: {visted_patents}")
        break